# Reconnaissance de Pokemon par image

### Un peu de contexte

Le but de ce projet est de construire un réseau de neurones entièrement connecté capable de reconnaître un Pokemon à partir d'une image donnée.

L'implémentation est basée sur PyTorch et le modèle est entrainé et testé sur le dataset PokemonClassification : https://huggingface.co/datasets/keremberke/pokemon-classification.

In [1]:
import torch
from torch import nn
import copy
from torchvision.io import read_image
import numpy as np
import glob
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from torchvision import transforms

### Étape 1 : Charger les données

En PyTorch, les données doivent être transmise au réseau de neurones à l'aide d'un loader. La première étape est de créer une classe de type `Dataset` que la fonction `DataLoader` prend en argument. La classe doit au moins posséder les trois routines `__init__`, `__len__` et `__getitem__`.

L'implémentation de cette classe est inspirée du tp1.a, avec une légère modification du `__getitem__` afin d'adapter les dimensions et que le tensor cible soit un scalaire.

Pour information, toutes les images chargées sont échantillonnées pour un soucis de rapidité sans en altérer la performance du model.

Particularité pour le chargement du dataset : nous avons remarqué que l'ordre des pokemon n'étaient pas le même entre chaque partie du dataset (training, test, validation). De plus, certains pokemon sont présents dans la partie testing mais pas dans les données d'entraînement. 

Ainsi, il a fallu trouver un moyen de faire coincider les labels de tous les Pokemon peu importe la partie du dataset. Pour ce faire, une look-table globale est créée à partir du parcours du dataset entier afin de définir les class map de chaque dataloader à partir des même labels.

In [2]:
pokemon_table = {}
for e in ["test", "training", "validation"]:
    for i, class_path in enumerate(glob.glob("data/" + e + "/*")):
        class_name = class_path.split("/")[-1]
        if class_name not in pokemon_table.keys():
            pokemon_table[class_name] = i

In [3]:
class CoRDataset(Dataset):
    def __init__(self, path, transform=None):
        self.imgs_path = path
        self.class_map = {}
        self.transform = transform
        file_list = glob.glob(self.imgs_path + "*")
        self.data=[]
        self.img =[]
        for i, class_path in enumerate(file_list):
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
                img = plt.imread(img_path)
                img = img/np.amax(img)
                R, G, B = img[::4,::4,0], img[::4,::4,1], img[::4,::4,2]
                #img = 0.2989 * R + 0.5870 * G + 0.1140 * B # On passe l'image en niveaux de gris pour que ça soit plus rapide
                img = np.stack([R, G, B], 2)
                self.img.append(img)
            self.class_map[class_name] = pokemon_table[class_name]
        self.img_dim = (3, 56, 56)
    
    def __len__(self):
        return len(self.data)    

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(self.img[idx])
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor.float(), class_id

### Étape 2 : Construire son réseau de neurone

Nous avons décidé d'implémenter un réseau de neurone classique, en utilisant la fonction `nn.Sequential` qui prend comme argument la liste des couches avec leur nombre de neurones associés.

In [4]:
pokemonmodel = torch.nn.Sequential(torch.nn.Linear(3 * 56 * 56, 512),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(0.6),
                                       torch.nn.Linear(512, 256),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(0.6),
                                       torch.nn.Linear(256, 110),
                                       )

### Étape 3 :  choisir une fonction de coût et une méthode d'optimisation.

Dans notre cas nous prenons comme fonction de coût l'entreopie croisée et pour l'optimisation, nous utilisons la descende de gradient stochastique (SGD : *Stochastic Gradient Descent*). Nous verrons également plus loin une implémentation variant entre Adam, SGD et Adagrad.

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(pokemonmodel.parameters(), lr=0.0005, weight_decay=10**(-6))

### Étape 4 : création des loader d'entraintement, de validation et de test

Ici on choisit un batch de 200, le plus efficace (ratio temps/précision) au vu de nos tests.

In [6]:
BATCH_SIZE = 200
if __name__ == "__main__":
    training_set = CoRDataset("data/training/")
    training_loader = DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True)
    validation_set = CoRDataset("data/validation/")
    validation_loader = DataLoader(validation_set, shuffle=False)
    test_set = CoRDataset("data/test/")
    test_loader = DataLoader(test_set, shuffle=False)

On définit ensuite la fonction d'entrainement `Learning` ainsi qu'une fonction utilitaire d'affichage de performance `Print_loss_accuracy`.
Cette fonction sont inspirées de l'implémentation du TP1.a à l'exception de la méthode `view` qui, ici, ne prend pas la batchsize car on souhaite "aplatir" l'image (flatten). 

In [7]:
def Print_loss_accuracy(nepoch, tloss, vloss, accuracy, best_tloss, best_vloss, best_accuracy):
    print ("{:<6} {:<15} {:<17} {:<15} {:<20} {:<22} {:<15}".format(nepoch, tloss, vloss, accuracy, best_tloss, best_vloss, best_accuracy))

In [8]:
def Learning(nepoch, model, crit, optim, batchsize, trainingloader, validationloader, algo, learning_rate, decay):
    writer = SummaryWriter(log_dir='runs/' + str(algo) + ': lr=' + str(learning_rate)+', decay = ' + str(decay))
    best_tloss = 100.
    best_vloss = 100.
    best_accuracy = 0.
    Print_loss_accuracy('Epoch', 'training loss', 'validation loss', 'accuracy', 'best train loss',
                        'best validation loss', 'best accuracy')

    for nepoch in range(nepoch):
        tloss = 0.
        vloss = 0.
        correct_test = 0
        model.train()

        for images, labels in trainingloader:
            optim.zero_grad()
            #print(images.size(0))
            #print(images.view(batchsize, -1).size())
            predicted = model(images.view(images.size(0), -1))
            loss = crit(predicted.squeeze(), labels.squeeze())
            loss.backward()
            optim.step()
            tloss += loss.item() * images.size(0)

        tloss /= len(trainingloader.dataset)

        model.eval()

        for images, labels in validationloader:
            predicted = model(images.view(1, -1))
            loss = crit(predicted.squeeze(), labels.squeeze())
            correct_test += (predicted.argmax(1) == labels).sum().item()
            vloss += loss.item() * images.size(0)

        vloss /= len(validationloader.dataset)
        accuracy = 100 * correct_test / len(validationloader.dataset)
        writer.add_scalar('Accuracy', accuracy, nepoch)
        writer.add_scalar('Loss/test', tloss, nepoch)
        writer.add_scalar('Loss/validation', vloss, nepoch)

        if accuracy >= best_accuracy:
            torch.save(model, "best_model.pth")
            best_accuracy = accuracy
        if vloss <= best_vloss:
            best_vloss = vloss
        if tloss <= best_tloss:
            best_tloss = tloss

        Print_loss_accuracy(nepoch + 1,
                            np.round(tloss, 8),
                            np.round(vloss, 8),
                            np.round(accuracy, 8),
                            np.round(best_tloss, 8),
                            np.round(best_vloss, 8),
                            np.round(best_accuracy, 8))
        writer.close()

### Let's train !!!

Il ne reste plus qu'à entrainer le modèle à partir des Dataloader précédemment crées !

On décide ici d'effectuer 50 epochs, à ajuster selon besoin.

**La fonction d'entrainement enregistre les performances de chaque itération sur un Tensorboard dont les logs sont enregistrés sous le dossier `runs`.**

In [9]:
Learning(50, pokemonmodel, criterion, optimizer, BATCH_SIZE, training_loader, validation_loader, "SGD", 0.005, 10**(-6))

Epoch  training loss   validation loss   accuracy        best train loss      best validation loss   best accuracy  
1      4.71397593      4.7146829         0.0             4.71397593           4.7146829              0.0            
2      4.71078858      4.71459272        0.0             4.71078858           4.71459272             0.0            
3      4.70966286      4.71408475        0.07194245      4.70966286           4.71408475             0.07194245     
4      4.70859163      4.71353704        0.0             4.70859163           4.71353704             0.07194245     
5      4.70616248      4.71359661        0.0             4.70616248           4.71353704             0.07194245     
6      4.70372557      4.7129276         0.07194245      4.70372557           4.7129276              0.07194245     
7      4.70671889      4.71262293        0.14388489      4.70372557           4.71262293             0.14388489     
8      4.70613202      4.7122704         0.07194245      4.70372

KeyboardInterrupt: 

### Optimisons tout ça !!

Maintenant que notre réseau de neurones est fonctionnel, il est temps d'optimiser ses performances.

Pour cela, on cherche à faire varier :
- La fonction d'activation (entre SGD, Adam, Adagrad).
- Le weight decay (dont le but est d'ajouter une pénalité au model en cas d'erreur).
- Le learning rate (qui détermine la taille du pas à chaque itération).

Pour ne pas biaiser l'apprentissage entre 2 itérations, le modèle est reinitialisé à chaque entraînement via `copy.deepcopy`.

Pour un soucis de rapidité, le nombre d'epoch ainsi que chaque choix d'itérations peut être modifié comme souhaité.

**Ces itérations sur différentes valeurs est particulièrement utile afin de se rendre compte des hyperparamètres optimaux. (Voir rapport pour courbes et analyse de performance)**

In [ ]:
initial_model = copy.deepcopy(pokemonmodel)
for opti in ["SGD", "Adam", "Adagrad"]:
    for val in [0, 10 ** (-5), 10 ** (-2), 10 ** 0, 10 ** 2]:
        for learning_rate in [0.01, 0.001, 0.005]:
            pokemonmodel = copy.deepcopy(initial_model)
            optimizer = torch.optim.Adam(pokemonmodel.parameters(), lr=learning_rate, weight_decay=val)
            if opti == "SGD":
                optimizer = torch.optim.SGD(pokemonmodel.parameters(), lr=learning_rate, weight_decay=val)
            elif opti == "Adagrad":
                optimizer = torch.optim.Adagrad(pokemonmodel.parameters(), lr=learning_rate, weight_decay=val)
            Learning(25, pokemonmodel, criterion, optimizer, BATCH_SIZE, training_loader, validation_loader, opti, learning_rate, val)